In [1]:
import torch
import random
import numpy as np
import argparse

import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
from utils import train, test
from models import GCN, GAT, LP

In [2]:
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
pubmed = Planetoid(root='.', name='Pubmed')
torch.use_deterministic_algorithms(True)

In [3]:
k = 1
seeds = [42, 2021, 1234]
lr = 0.05
epochs = 200

GAT/GCN

In [4]:
dataset = citeseer
model = GCN(dataset.num_features, 16, dataset.num_classes)

# dataset = cora
# model = GCN(dataset.num_features, 16, dataset.num_classes)

# dataset = pubmed
# model = GCN(dataset.num_features, 16, dataset.num_classes)

# dataset = citeseer
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

# dataset = cora
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

# dataset = pubmed
# model = GAT(dataset.num_features, 8, dataset.num_classes, heads=8)

In [5]:
torch.manual_seed(0)
data = dataset[0]
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False

In [6]:
av_val_acc = av_test_acc = 0
state_dict = model.state_dict().copy()

for seed in seeds:
    print("RUNNING FOR SEED =", seed)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    
    model.load_state_dict(state_dict)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

    best_val_acc = final_test_acc = 0
    for epoch in range(1, 200):
        loss = train(model, data, optimizer, scheduler=None, loss='cross_entropy')
        train_acc, val_acc, tmp_test_acc = test(model, data)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        if epoch % 25 == 0:
            log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
    print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')
    av_val_acc += best_val_acc
    av_test_acc += test_acc
    
print(f'Average Val Acc / Average Test Acc: {av_val_acc / len(seeds):.4f} / {av_test_acc / len(seeds):.4f}')

RUNNING FOR SEED = 42
Epoch: 025, Loss: 3.2782238577055978e-06, Train: 1.0000, Val: 0.3080, Test: 0.3480
Epoch: 050, Loss: 6.009629214531742e-05, Train: 1.0000, Val: 0.3800, Test: 0.3480
Epoch: 075, Loss: 0.0004190208564978093, Train: 1.0000, Val: 0.3940, Test: 0.3480
Epoch: 100, Loss: 0.0008498384268023074, Train: 1.0000, Val: 0.3980, Test: 0.3480
Epoch: 125, Loss: 0.0009150051628239453, Train: 1.0000, Val: 0.4160, Test: 0.3700
Epoch: 150, Loss: 0.0009999716421589255, Train: 1.0000, Val: 0.4240, Test: 0.3830
Epoch: 175, Loss: 0.0009768542367964983, Train: 1.0000, Val: 0.4340, Test: 0.3780
Best Val Acc: 0.4400 Test Acc: 0.3730
RUNNING FOR SEED = 2021
Epoch: 025, Loss: 0.0012220707722008228, Train: 1.0000, Val: 0.3780, Test: 0.3440
Epoch: 050, Loss: 0.0009002771112136543, Train: 1.0000, Val: 0.3700, Test: 0.3440
Epoch: 075, Loss: 0.0008866720017977059, Train: 1.0000, Val: 0.3740, Test: 0.3440
Epoch: 100, Loss: 0.000879831553902477, Train: 1.0000, Val: 0.3700, Test: 0.3440
Epoch: 125, Lo

LPA

In [7]:
av_val_acc = av_test_acc = 0

for seed in seeds:
    print("RUNNING FOR SEED =", seed)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    best_val_acc = final_test_acc = 0
    best_l = best_a = 0

    for l in [1, 2, 4, 8, 16, 32]:
        for a in [0.05, 0.1, 0.3, 0.6, 0.8, 0.9, 0.95, 0.99, 1]:
            model = LP(num_layers=l, alpha=a)
            outs = model.train(dataset)
            train_acc, val_acc, tmp_test_acc = model.test()
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                test_acc = tmp_test_acc
                best_l = l
                best_a = a
    print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}', f'Best l: {best_l}', f'Best a: {best_a}')
    av_test_acc += test_acc
    av_val_acc += best_val_acc    
print(f'Average Val Acc / Average Test Acc: {av_val_acc / len(seeds):.4f} / {av_test_acc / len(seeds):.4f}')

RUNNING FOR SEED = 42
Best Val Acc: 0.3040 Test Acc: 0.2990 Best l: 16 Best a: 0.05
RUNNING FOR SEED = 2021
Best Val Acc: 0.3040 Test Acc: 0.2990 Best l: 16 Best a: 0.05
RUNNING FOR SEED = 1234
Best Val Acc: 0.3040 Test Acc: 0.2990 Best l: 16 Best a: 0.05
Average Val Acc / Average Test Acc: 0.3040 / 0.2990
